In [1]:
import json
import pandas as pd

In [2]:
ridings = pd.read_csv("../data/shp/intersect/ridings_2021-2023.csv")

# Compute Scale_factor to convert 2021 riding -> 2023 riding
# Scale_factor is based on 2021 population density and size of overlapping area
ridings["Scale_factor"] = ridings["Intersect_Area"] * ridings["2021_POP_CNT"] / ridings["2021_SHAPE_area"]

# Normalize Scale_factor so that it sums to 1.0 for each riding
ridings["Subtotal"] = ridings.groupby("2023_FED_NUM")["Scale_factor"].transform("sum")
ridings["Scale_factor"] = ridings["Scale_factor"] / ridings["Subtotal"]

ridings = ridings.drop(columns="Subtotal")

In [3]:
ridings.sort_values(by="2023_FED_NUM")

,2021_FED_NUM,2021_ED_NAMEE,2021_SHAPE_area,2023_FED_NUM,2023_ED_NAMEE,2023_POP_CNT,2023_PROV,2023_Shape_area,Intersect_Area,2021_POP_CNT,Scale_factor
22,10007,St. John's South--Mount Pearl,6.214225e+08,10001,Avalon,81781,NL,8.020016e+09,4.452072e+08,81544,0.464243
12,10001,Avalon,9.806182e+09,10001,Avalon,81781,NL,8.020016e+09,7.408623e+09,87191,0.523464
19,10006,St. John's East,9.755155e+08,10001,Avalon,81781,NL,8.020016e+09,1.727750e+07,87345,0.012293
23,10007,St. John's South--Mount Pearl,6.214225e+08,10002,Cape Spear,85038,NL,1.952738e+08,1.608033e+08,81544,1.000000
15,10002,Bonavista--Burin--Trinity,3.561748e+10,10003,Central Newfoundland,74122,NL,5.753482e+10,1.791981e+08,71898,0.004869
...,...,...,...,...,...,...,...,...,...,...,...
239,59041,Victoria,4.265028e+08,59042,Victoria,123482,BC,4.312378e+08,4.244088e+08,123482,1.000000
240,59042,West Vancouver--Sunshine Coast--Sea to Sky Cou...,1.475831e+10,59043,West Vancouver—Sunshine Coast—Sea to Sky Country,114257,BC,1.512751e+10,1.486703e+10,131206,0.513134
235,59039,Vancouver Quadra,5.750324e+07,59043,West Vancouver—Sunshine Coast—Sea to Sky Country,114257,BC,1.512751e+10,1.945420e+06,109328,0.014360
145,59031,Steveston--Richmond East,2.308914e+08,59043,West Vancouver—Sunshine Coast—Sea to Sky Country,114257,BC,1.512751e+10,1.160892e+08,102230,0.199550


In [4]:
ridings

,2021_FED_NUM,2021_ED_NAMEE,2021_SHAPE_area,2023_FED_NUM,2023_ED_NAMEE,2023_POP_CNT,2023_PROV,2023_Shape_area,Intersect_Area,2021_POP_CNT,Scale_factor
0,24065,Marc-Aurèle-Fortin,5.757633e+07,24045,Marc-Aurèle-Fortin,104636,QC,5.757633e+07,5.593495e+07,104636,1.000000
1,24066,Saint-Hyacinthe--Bagot,1.948976e+09,24066,Saint-Hyacinthe—Bagot—Acton,105086,QC,1.949183e+09,1.894367e+09,105086,1.000000
2,24067,Saint-Jean,7.342893e+08,24067,Saint-Jean,114617,QC,7.342892e+08,7.110450e+08,114617,1.000000
3,24069,Saint-Léonard--Saint-Michel,2.086278e+07,24069,Saint-Léonard—Saint-Michel,112922,QC,1.980223e+07,1.923424e+07,115553,1.000000
4,24069,Saint-Léonard--Saint-Michel,2.086278e+07,24026,Hochelaga—Rosemont-Est,110039,QC,2.425269e+07,1.530597e+05,115553,0.007999
...,...,...,...,...,...,...,...,...,...,...,...
642,35108,Toronto Centre,6.161116e+06,35112,University—Rosedale,123812,ON,1.451405e+07,3.411924e+05,119901,0.063173
643,35109,Toronto--Danforth,2.766616e+07,35110,Toronto—Danforth,105472,ON,2.784525e+07,2.637669e+07,105472,0.989564
644,35110,University--Rosedale,1.427508e+07,35111,Toronto—St. Paul's,125438,ON,1.630702e+07,1.376973e+06,106216,0.084481
645,35110,University--Rosedale,1.427508e+07,35109,Toronto Centre,121703,ON,6.735959e+06,3.467437e+05,106216,0.023068


In [6]:
factors = {}

for row in ridings.itertuples():
    id_2023 = row._4
    id_2021 = row._1
    
    if id_2023 not in factors:
        factors[id_2023] = {}
    
    factors[id_2023].update({id_2021: round(row.Scale_factor, 5)})

In [9]:
result = []

for id_2023, factor in factors.items():
    result.append({
        "id_2023": id_2023,
        "factors": [{"id_2021": id_2021, "factor": fact} for id_2021, fact in factor.items()]
    })

In [10]:
result

[{'id_2023': 24045, 'factors': [{'id_2021': 24065, 'factor': 1.0}]},
 {'id_2023': 24066, 'factors': [{'id_2021': 24066, 'factor': 1.0}]},
 {'id_2023': 24067, 'factors': [{'id_2021': 24067, 'factor': 1.0}]},
 {'id_2023': 24069, 'factors': [{'id_2021': 24069, 'factor': 1.0}]},
 {'id_2023': 24026,
  'factors': [{'id_2021': 24069, 'factor': 0.008},
   {'id_2021': 24028, 'factor': 0.992}]},
 {'id_2023': 24027,
  'factors': [{'id_2021': 24069, 'factor': 0.04563},
   {'id_2021': 24029, 'factor': 0.95437}]},
 {'id_2023': 24070, 'factors': [{'id_2021': 24070, 'factor': 1.0}]},
 {'id_2023': 24007,
  'factors': [{'id_2021': 24071, 'factor': 0.78883},
   {'id_2021': 24074, 'factor': 0.21117}]},
 {'id_2023': 24017,
  'factors': [{'id_2021': 24071, 'factor': 0.21069},
   {'id_2021': 24021, 'factor': 0.78931}]},
 {'id_2023': 59011,
  'factors': [{'id_2021': 59011, 'factor': 0.93699},
   {'id_2021': 59030, 'factor': 0.01082},
   {'id_2021': 59033, 'factor': 0.05219}]},
 {'id_2023': 59013,
  'factors':

In [11]:
with open("../outputs/riding_conv_2021-2023.json", "w") as f:
    json.dump(result, f, indent=4)